In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import time
import re

In [31]:
#取地今日時間

from datetime import datetime
from dateutil.relativedelta import relativedelta
n_days = datetime.now() + relativedelta(months=6)
print (n_days.strftime('%Y/%m/%d')) # %H:%M:%S'))

2019/03/07


In [32]:
#建立資料庫引擎

from sqlalchemy import create_engine
import pymysql
engine = create_engine('mysql+pymysql://root@127.0.0.1:3306/tour?charset=utf8')

In [33]:
#定義 orangeres 方法  取得orange旅行社網頁資料

def orangeres (p = 1):

    url = 'http://www.orangetour.com.tw/EW/Services/SearchListData.asp'
    page = p
    #headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
    payload={
        'displayType': 'G',
    #    'subCd': 
        'orderCd': '1',
        'pageALL': str(page),
        'pageGO': '1',
        'pagePGO': '1',
        'waitData': 'false',
        'waitPage': 'false',
    #    'mGrupCd': 
    #    'SrcCls': 
    #    'tabList':
    #    'regmCd': 
    #    'regsCd': 
        'beginDt': str(datetime.now().strftime('%Y/%m/%d')),
        'endDt': '9999/12/31',#str(n_days.strftime('%Y/%m/%d')),
    #    'portCd': 
    #    'tdays': 
    #    'bjt': 
    #    'carr': 
        'allowJoin': '1',
        'allowWait': '1'
    #    'ikeyword': 
    }
    rs = requests.session()
    res = rs.post(url,data = payload)

    res.encoding = 'utf-8'
    return res

In [34]:
# 定義 orangefp 方法  取得當日網站資料總數

def orangefp():
    url2 = 'http://www.orangetour.com.tw/EW/GO/GroupList.asp'
    
    res2 = requests.get(url2,'html.parser')
    res2.encoding = 'utf-8'
    soup = BeautifulSoup(res2.text,'html.parser')
    a = soup.find_all('ul',{'class':'pagination'})[0].find_all('li',{'class':'next'})[-1].find('a')['onclick']
    fp = re.search(r'\d+',a)
    return int(fp.group(0))

In [35]:
#getfp()

In [36]:
#定義 orange() 方法  用迴圈抓取所有網頁資料 並轉出DataFrame 表單

def orange():
    data2 = pd.DataFrame()
    p = 1
    total = 0
    fp = orangefp()
    print(fp)
    while True:
        res = orangeres(p)
        d = json.loads(res.text)
        d = d['All']

        for dd in d:
            df = pd.DataFrame([dd.values()])
            data2 = pd.concat([data2,df])
            
        total += len(d)
        print((p,total),end=",")
        

        if p == fp:        
            break
            
        p += 1
        time.sleep(2)
    data2.columns = (d[0].keys())
    data2['SignUpLink'] = data2['SignUpLink'].apply(lambda x:x['Name'])
    data2.reset_index(inplace = True,drop = True)
    return data2

In [37]:
data2 = orange()
data2

24
(1, 20),(2, 40),(3, 60),(4, 80),(5, 100),(6, 120),(7, 140),(8, 160),(9, 180),(10, 200),(11, 220),(12, 240),(13, 260),(14, 280),(15, 300),(16, 320),(17, 340),(18, 360),(19, 380),(20, 400),(21, 420),(22, 440),(23, 460),(24, 476),

,GrupCd,LeavDt,WeekDay,GrupSnm,ItnRk3S,EstmYqt,DoneYqt,SaleYqt,DordQt,EstmTotqt,...,SaleAm,AgtAm,GrupLn,SubCd,SubCdAnm,SortSq,Url,ShareUrl,SrcCls,ImgUrl
0,ME1080908TKA,2018/09/08,六,直飛土耳其世界遺產、愛琴海蕃紅花10日(5晚5星+1晚洞穴),,26,0,0,0,26,...,48900,48900,10,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=ME1080908TKA,http://www.orangetour.com.tw/EW/GO/GroupDetail...,0,/eWeb_asiantour/IMGDB/000103/000229/00004137.JPG
1,AK0880908KQA,2018/09/08,六,肯亞 我要玩狠大8天,,13,0,12,0,13,...,125900,125900,8,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=AK0880908KQA,http://www.orangetour.com.tw/EW/GO/GroupDetail...,0,/eweb_asiantour/images/default_demo.jpg
2,EW1080909QRA,2018/09/09,日,品味法拉利 南北義 龐貝 藍洞 OUTLET 10天,,26,0,25,0,26,...,75900,75900,10,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=EW1080909QRA,http://www.orangetour.com.tw/EW/GO/GroupDetail...,0,/eWeb_asiantour/IMGDB/000000/00005642.JPG
3,ES1080909CAA,2018/09/09,日,浪漫愛琴海雙島：聖托里尼、米克諾斯、天空之城、諸神的故鄉、希臘10天之旅,,16,16,0,15,16,...,77900,77900,10,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=ES1080909CAA,http://www.orangetour.com.tw/EW/GO/GroupDetail...,0,/eweb_asiantour/images/default_demo.jpg
4,EW1080901BRA,2018/09/10,一,Hello! 長榮荷德比法 3遊船 OUTLET 10天,,26,0,0,0,26,...,69900,69900,10,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=EW1080901BRA,http://www.orangetour.com.tw/EW/GO/GroupDetail...,0,/eweb_asiantour/images/default_demo.jpg
5,ICN180912KEB,2018/09/12,三,親子水陸雙樂園5日,,16,0,16,0,16,...,13000,13000,5,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=ICN180912KEB,http://www.orangetour.com.tw/EW/GO/GroupDetail...,0,/eWeb_asiantour/IMGDB/000126/00003008.JPG
6,ICN180912KEA,2018/09/12,三,哈韓94狂～超人氣三大主題5日,,16,0,16,0,16,...,15000,15000,5,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=ICN180912KEA,http://www.orangetour.com.tw/EW/GO/GroupDetail...,0,/eweb_asiantour/images/default_demo.jpg
7,TYO180913ITA,2018/09/13,四,東京蝦拼半自助漫遊五日,,16,0,15,0,16,...,21999,21999,5,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=TYO180913ITA,http://www.orangetour.com.tw/EW/GO/GroupDetail...,0,/eWeb_asiantour/IMGDB/000002/000003/00000380.JPG
8,AK1080913KQA,2018/09/13,四,肯亞 野性呼喚10天 非洲王子系列,,13,0,12,0,13,...,159900,159900,10,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=AK1080913KQA,http://www.orangetour.com.tw/EW/GO/GroupDetail...,0,/eweb_asiantour/images/default_demo.jpg
9,EE1080914BRA,2018/09/14,五,奧捷雙國、米其林、湖區、古堡、遊船、雙登塔10天(長榮直飛),,26,0,25,0,26,...,62900,62900,10,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=EE1080914BRA,http://www.orangetour.com.tw/EW/GO/GroupDetail...,0,/eweb_asiantour/images/default_demo.jpg


In [38]:
#檢查欄位屬性
data2.dtypes

GrupCd             object
LeavDt             object
WeekDay            object
GrupSnm            object
ItnRk3S            object
EstmYqt             int64
DoneYqt             int64
SaleYqt             int64
DordQt              int64
EstmTotqt           int64
PortNm             object
FullSts            object
RuleDr             object
SaleCd              int64
HotTp              object
GordLk               bool
OrderDl            object
HtlOkFg              bool
AdvFg                bool
GuaranteeFg          bool
PromoteFg            bool
ApplyQt             int64
DepositQt           int64
NoDepositQt         int64
RqstQt              int64
SignUpLink         object
IsShowGuarantee      bool
IsShowPromote        bool
IsShowHotTp          bool
HotTpNm            object
RecCnt              int64
GoCnt               int64
PgoCnt              int64
RowId               int64
SacctNo            object
MgrupCd            object
SaleAm              int64
AgtAm               int64
GrupLn      

In [39]:
#檢查欄位內容
data2.loc[0]

GrupCd                                                  ME1080908TKA
LeavDt                                                    2018/09/08
WeekDay                                                            六
GrupSnm                               直飛土耳其世界遺產、愛琴海蕃紅花10日(5晚5星+1晚洞穴)
ItnRk3S                                                             
EstmYqt                                                           26
DoneYqt                                                            0
SaleYqt                                                            0
DordQt                                                             0
EstmTotqt                                                         26
PortNm                                                      台北-桃園機場 
FullSts                                                             
RuleDr                                                              
SaleCd                                                             2
HotTp                             

In [40]:
#將資料依需求篩選欄位，並將欄位名稱改為旅遊咖網頁使用名稱，最後將結果存入mysql

date2 = data2[['GrupCd','LeavDt','GrupLn','WeekDay','PortNm']]
date2.rename(columns={'GrupCd':'group_code','LeavDt':'date','GrupLn':'days'},inplace = True)
date2.to_sql(name='date2',con=engine,  if_exists = 'replace', index=False)
date2.head()

e:\anacondax64\envs\py36\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,group_code,date,days,WeekDay,PortNm
0,ME1080908TKA,2018/09/08,10,六,台北-桃園機場
1,AK0880908KQA,2018/09/08,8,六,台北-桃園機場
2,EW1080909QRA,2018/09/09,10,日,台北-桃園機場
3,ES1080909CAA,2018/09/09,10,日,台北-桃園機場
4,EW1080901BRA,2018/09/10,10,一,台北-桃園機場


In [41]:
sale2 = data2[['GrupCd','SaleYqt','EstmTotqt','SaleAm']]
sale2.rename(columns={'GrupCd':'group_code','SaleYqt':'sell','EstmTotqt':'qty','SaleAm':'minprice'},inplace = True)
sale2.to_sql(name='sale2',con=engine,  if_exists = 'replace', index=False)
sale2.head()

e:\anacondax64\envs\py36\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,group_code,sell,qty,minprice
0,ME1080908TKA,0,26,48900
1,AK0880908KQA,12,13,125900
2,EW1080909QRA,25,26,75900
3,ES1080909CAA,0,16,77900
4,EW1080901BRA,0,26,69900


In [42]:
base2 = data2[['GrupSnm','GrupCd','SubCdAnm','SignUpLink','ImgUrl']]
base2['tour'] = 'orange'
base2.rename(columns={'GrupSnm':'title','GrupCd':'group_code','tour':'agency_code','ImgUrl':'pic'},inplace = True)
base2.to_sql(name='base2',con=engine,  if_exists = 'replace', index=False)
base2.head()

e:\anacondax64\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
e:\anacondax64\envs\py36\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,title,group_code,SubCdAnm,SignUpLink,pic,agency_code
0,直飛土耳其世界遺產、愛琴海蕃紅花10日(5晚5星+1晚洞穴),ME1080908TKA,團,候補,/eWeb_asiantour/IMGDB/000103/000229/00004137.JPG,orange
1,肯亞 我要玩狠大8天,AK0880908KQA,團,報名,/eweb_asiantour/images/default_demo.jpg,orange
2,品味法拉利 南北義 龐貝 藍洞 OUTLET 10天,EW1080909QRA,團,截止,/eWeb_asiantour/IMGDB/000000/00005642.JPG,orange
3,浪漫愛琴海雙島：聖托里尼、米克諾斯、天空之城、諸神的故鄉、希臘10天之旅,ES1080909CAA,團,請電洽,/eweb_asiantour/images/default_demo.jpg,orange
4,Hello! 長榮荷德比法 3遊船 OUTLET 10天,EW1080901BRA,團,候補,/eweb_asiantour/images/default_demo.jpg,orange


In [ ]:
#############################分隔線  ##############################

In [43]:
def gloriares (p = 1):

    url = 'https://www.gloriatour.com.tw/EW/Services/SearchListData.asp'
    page = p
    #headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
    payload={
        'displayType': 'G',
    #    'subCd': 
        'orderCd': '1',
        'pageALL': str(page),
        'pageGO': '1',
        'pagePGO': '1',
        'waitData': 'false',
        'waitPage': 'false',
    #    'mGrupCd': 
    #    'SrcCls': 
    #    'tabList': 
    #    'regmCd': 
    #    'regsCd': 
        'beginDt': str(datetime.now().strftime('%Y/%m/%d')),
        'endDt': '9999/12/31',#str(n_days.strftime('%Y/%m/%d')),
    #    'portCd': 
    #    'tdays': 
    #    'bjt': 
    #    'carr': 
        'allowJoin': '1',
        'allowWait': '1'
    #    'ikeyword': 
    }
    rs = requests.session()
    res = rs.post(url,data = payload)

    res.encoding = 'big5'
    return res

In [44]:
def gloriafp():
    url2 = 'https://www.gloriatour.com.tw/EW/GO/GroupList.asp'
    
    res2 = requests.get(url2,'html.parser')
    res2.encoding = 'utf-8'
    soup = BeautifulSoup(res2.text,'html.parser')
    a = soup.find_all('ul',{'class':'pagination'})[0].find_all('li',{'class':'next'})[-1].find('a')['onclick']
    fp = re.search(r'\d+',a)
    return int(fp.group(0))

In [45]:
gloriafp()

144

In [46]:
def gloria():
    data2 = pd.DataFrame()
    p = 1
    n=0
    total = 0
    fp = gloriafp()
    while True:
        res = gloriares(p)
        d = json.loads(res.text)
        d = d['All']
        
        for dd in d:
            df = pd.DataFrame([dd.values()])
            data2 = pd.concat([data2,df])
            
        total += len(d)
        print((p,total),end=",")
        
        if p == fp:
            break
            
        p += 1
        n += 1
        time.sleep(2)
    data2.columns = (d[0].keys())
    data2['SignUpLink'] = data2['SignUpLink'].apply(lambda x:x['Name'])
    data2.reset_index(inplace = True,drop = True)
    return data2

In [47]:
data1 = gloria()
data1

(1, 20),(2, 40),(3, 60),(4, 80),(5, 100),(6, 120),(7, 140),(8, 160),(9, 180),(10, 200),(11, 220),(12, 240),(13, 260),(14, 280),(15, 300),(16, 320),(17, 340),(18, 360),(19, 380),(20, 400),(21, 420),(22, 440),(23, 460),(24, 480),(25, 500),(26, 520),(27, 540),(28, 560),(29, 580),(30, 600),(31, 620),(32, 640),(33, 660),(34, 680),(35, 700),(36, 720),(37, 740),(38, 760),(39, 780),(40, 800),(41, 820),(42, 840),(43, 860),(44, 880),(45, 900),(46, 920),(47, 940),(48, 960),(49, 980),(50, 1000),(51, 1020),(52, 1040),(53, 1060),(54, 1080),(55, 1100),(56, 1120),(57, 1140),(58, 1160),(59, 1180),(60, 1200),(61, 1220),(62, 1240),(63, 1260),(64, 1280),(65, 1300),(66, 1320),(67, 1340),(68, 1360),(69, 1380),(70, 1400),(71, 1420),(72, 1440),(73, 1460),(74, 1480),(75, 1500),(76, 1520),(77, 1540),(78, 1560),(79, 1580),(80, 1600),(81, 1620),(82, 1640),(83, 1660),(84, 1680),(85, 1700),(86, 1720),(87, 1740),(88, 1760),(89, 1780),(90, 1800),(91, 1820),(92, 1840),(93, 1860),(94, 1880),(95, 1900),(96, 1920),(97, 1

,GrupCd,LeavDt,WeekDay,GrupSnm,ItnRk3S,EstmYqt,DoneYqt,SaleYqt,DordQt,EstmTotqt,...,SaleAm,AgtAm,GrupLn,SubCd,SubCdAnm,SortSq,Url,ShareUrl,SrcCls,ImgUrl
0,SELB5BX8907A,2018/09/07,五,【時尚玩釜山】∼升等五花飯店∼佛國寺、八公山纜車、西面鬧區、天空步道、甘川洞文化村５日,,18,0,17,0,18,...,12500,12500,5,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=SELB5BX8907A,https://www.gloriatour.com.tw/EW/GO/GroupDetai...,0,/eWeb_gloriatour/IMGDB/000012/000013/000589/00...
1,SELA5OZ8907D,2018/09/07,五,【秒殺！玩首爾】★不走人蔘護肝寶★明洞半天自由行、樂天超市、ＡＲ美術館、弘大藝術街、夜燈節５日,,18,0,17,0,18,...,15400,15400,5,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=SELA5OZ8907D,https://www.gloriatour.com.tw/EW/GO/GroupDetai...,0,/eWeb_gloriatour/IMGDB/000012/000013/000589/00...
2,SIND4CI8907A,2018/09/07,五,【線上旅展】就去新加坡玩吧自由輕鬆走四日遊【送環球影城門票＋園區餐券】(二人成行),,6,0,6,0,6,...,16500,16500,4,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=SIND4CI8907A,https://www.gloriatour.com.tw/EW/GO/GroupDetai...,0,/eWeb_gloriatour/IMGDB/000012/000014/000811/00...
3,CJU05IT8907A,2018/09/07,五,【夏艷濟州】韓國濟州奇幻山坡、泰迪熊野生動物王國、透明獨木舟、樂高樂園、塗鴉秀５日,,24,0,0,0,24,...,18500,18500,5,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=CJU05IT8907A,https://www.gloriatour.com.tw/EW/GO/GroupDetai...,0,/eWeb_gloriatour/IMGDB/000012/000013/000589/00...
4,SINI4CI8907A,2018/09/07,五,【線上旅展】星光熠熠新加坡環球影城濱海灣花園金沙娛樂城超值四日(四人成行),,6,0,6,0,6,...,21000,21000,4,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=SINI4CI8907A,https://www.gloriatour.com.tw/EW/GO/GroupDetai...,0,/eWeb_gloriatour/IMGDB/000012/000014/000811/00...
5,KS205CI8907A,2018/09/07,五,【馬新旺福】馬新雙國吉隆坡高塔自助餐馬六甲5D立體藝術館金沙娛樂城超值五日遊(新加坡一晚),,18,0,17,0,18,...,21588,21588,5,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=KS205CI8907A,https://www.gloriatour.com.tw/EW/GO/GroupDetai...,0,/eWeb_gloriatour/IMGDB/000012/000014/000763/00...
6,BKIA5MH8907A,2018/09/07,五,【線上旅展】馬來西亞沙巴長鼻猴螢河遊泛舟海洋國家公園五日(4+5星)(6人成行),,8,0,8,0,8,...,25500,25500,5,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=BKIA5MH8907A,https://www.gloriatour.com.tw/EW/GO/GroupDetai...,0,/eWeb_gloriatour/IMGDB/000012/000014/000763/00...
7,SINA4CI8907A,2018/09/07,五,【米其林一星】食尚樂活新加坡環球影城S.E.A.海洋館濱海灣花園世界最高摩天輪四日【小孩不佔...,,18,0,17,0,18,...,26300,26300,4,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=SINA4CI8907A,https://www.gloriatour.com.tw/EW/GO/GroupDetai...,0,/eWeb_gloriatour/IMGDB/000012/000014/000811/00...
8,PPSD5PR8907A,2018/09/07,五,菲航直飛菲常渴望巴拉望一島一飯店超級玩家五日遊(2人成行),,20,0,0,0,20,...,26500,26500,5,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=PPSD5PR8907A,https://www.gloriatour.com.tw/EW/GO/GroupDetai...,0,/eWeb_gloriatour/IMGDB/000012/000014/000844/00...
9,PPSA5PR8907A,2018/09/07,五,菲航直飛菲常渴望巴拉望新尊爵五日遊(兩晚渡假村)(2人成行),,20,2,0,2,20,...,30900,30900,5,GO,團,7,/EW/GO/GroupDetail.asp?prodCd=PPSA5PR8907A,https://www.gloriatour.com.tw/EW/GO/GroupDetai...,0,/eWeb_gloriatour/IMGDB/000012/000014/000844/00...


In [48]:
data1.loc[0]

GrupCd                                                  SELB5BX8907A
LeavDt                                                    2018/09/07
WeekDay                                                            五
GrupSnm                  【時尚玩釜山】∼升等五花飯店∼佛國寺、八公山纜車、西面鬧區、天空步道、甘川洞文化村５日
ItnRk3S                                                             
EstmYqt                                                           18
DoneYqt                                                            0
SaleYqt                                                           17
DordQt                                                             0
EstmTotqt                                                         18
PortNm                                                      台北-桃園機場 
FullSts                                                             
RuleDr                                                              
SaleCd                                                             1
HotTp                             

In [49]:
date1 = data1[['GrupCd','LeavDt','GrupLn','WeekDay','PortNm']]
date1.rename(columns={'GrupCd':'group_code','LeavDt':'date','GrupLn':'days'},inplace = True)
date1.to_sql(name='date1',con=engine,  if_exists = 'replace', index=False)
date1.head()

e:\anacondax64\envs\py36\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,group_code,date,days,WeekDay,PortNm
0,SELB5BX8907A,2018/09/07,5,五,台北-桃園機場
1,SELA5OZ8907D,2018/09/07,5,五,台北-桃園機場
2,SIND4CI8907A,2018/09/07,4,五,台北-桃園機場
3,CJU05IT8907A,2018/09/07,5,五,台北-桃園機場
4,SINI4CI8907A,2018/09/07,4,五,台北-桃園機場


In [50]:
sale1 = data1[['GrupCd','SaleYqt','EstmTotqt','SaleAm']]
sale1.rename(columns={'GrupCd':'group_code','SaleYqt':'sell','EstmTotqt':'qty','SaleAm':'minprice'},inplace = True)
sale1.to_sql(name='sale1',con=engine,  if_exists = 'replace', index=False)
sale1.head()

e:\anacondax64\envs\py36\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,group_code,sell,qty,minprice
0,SELB5BX8907A,17,18,12500
1,SELA5OZ8907D,17,18,15400
2,SIND4CI8907A,6,6,16500
3,CJU05IT8907A,0,24,18500
4,SINI4CI8907A,6,6,21000


In [56]:
base1 = data1[['GrupSnm','GrupCd','SubCdAnm','SignUpLink','ImgUrl']]
base1['tour'] = 'orange'
base1.rename(columns={'GrupSnm':'title','GrupCd':'group_code','tour':'agency_code','ImgUrl':'pic'},inplace = True)
base1.to_sql(name='base1',con=engine,  if_exists = 'replace', index=False)
base1.head()

e:\anacondax64\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
e:\anacondax64\envs\py36\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,title,group_code,SubCdAnm,SignUpLink,pic,agency_code
0,【時尚玩釜山】∼升等五花飯店∼佛國寺、八公山纜車、西面鬧區、天空步道、甘川洞文化村５日,SELB5BX8907A,團,報名,/eWeb_gloriatour/IMGDB/000012/000013/000589/00...,orange
1,【秒殺！玩首爾】★不走人蔘護肝寶★明洞半天自由行、樂天超市、ＡＲ美術館、弘大藝術街、夜燈節５日,SELA5OZ8907D,團,報名,/eWeb_gloriatour/IMGDB/000012/000013/000589/00...,orange
2,【線上旅展】就去新加坡玩吧自由輕鬆走四日遊【送環球影城門票＋園區餐券】(二人成行),SIND4CI8907A,團,報名,/eWeb_gloriatour/IMGDB/000012/000014/000811/00...,orange
3,【夏艷濟州】韓國濟州奇幻山坡、泰迪熊野生動物王國、透明獨木舟、樂高樂園、塗鴉秀５日,CJU05IT8907A,團,候補,/eWeb_gloriatour/IMGDB/000012/000013/000589/00...,orange
4,【線上旅展】星光熠熠新加坡環球影城濱海灣花園金沙娛樂城超值四日(四人成行),SINI4CI8907A,團,報名,/eWeb_gloriatour/IMGDB/000012/000014/000811/00...,orange


In [ ]:
#########################  分隔線  ###################################

In [66]:
#設定資料庫連線

connect = pymysql.Connect(
#    host='localhost',
    host='127.0.0.1',
    port=3306,
    user='root',
    passwd='',
    db='tour',
    charset='utf8'
)
    
cursor = connect.cursor()

In [67]:
sql  =  ('SELECT * FROM tour.date2;')

In [68]:
#定義 getsqldata() 方法 ，方便操作SQL語法
def getsqldata():
    data = pd.read_sql(sql=sql,con=connect)
    return data

In [73]:
date2 =  getsqldata()
date2.head()

,group_code,sell,qty,minprice
0,ME1080908TKA,0,26,48900
1,AK0880908KQA,12,13,125900
2,EW1080909QRA,25,26,75900
3,ES1080909CAA,0,16,77900
4,EW1080901BRA,0,26,69900


In [74]:
#由 mysql 讀取資料
sql  =  ('SELECT * FROM tour.base2;')
base2 =  getsqldata()
base2.head()

,title,group_code,SubCdAnm,SignUpLink,pic,agency_code
0,直飛土耳其世界遺產、愛琴海蕃紅花10日(5晚5星+1晚洞穴),ME1080908TKA,團,候補,/eWeb_asiantour/IMGDB/000103/000229/00004137.JPG,orange
1,肯亞 我要玩狠大8天,AK0880908KQA,團,報名,/eweb_asiantour/images/default_demo.jpg,orange
2,品味法拉利 南北義 龐貝 藍洞 OUTLET 10天,EW1080909QRA,團,截止,/eWeb_asiantour/IMGDB/000000/00005642.JPG,orange
3,浪漫愛琴海雙島：聖托里尼、米克諾斯、天空之城、諸神的故鄉、希臘10天之旅,ES1080909CAA,團,請電洽,/eweb_asiantour/images/default_demo.jpg,orange
4,Hello! 長榮荷德比法 3遊船 OUTLET 10天,EW1080901BRA,團,候補,/eweb_asiantour/images/default_demo.jpg,orange


In [75]:
sql  =  ('SELECT * FROM tour.sale2;')
sale2 =  getsqldata()
sale2.head()

,group_code,sell,qty,minprice
0,ME1080908TKA,0,26,48900
1,AK0880908KQA,12,13,125900
2,EW1080909QRA,25,26,75900
3,ES1080909CAA,0,16,77900
4,EW1080901BRA,0,26,69900


In [81]:
#由 mysql 執行資料合併後讀取資料

sql  =  ("""
SELECT * 
FROM tour.base1 
INNER JOIN sale1 
ON base1.group_code = sale1.group_code;
""")
ddd =  getsqldata()
ddd

,title,group_code,SubCdAnm,SignUpLink,pic,agency_code,group_code,sell,qty,minprice
0,【時尚玩釜山】∼升等五花飯店∼佛國寺、八公山纜車、西面鬧區、天空步道、甘川洞文化村５日,SELB5BX8907A,團,報名,/eWeb_gloriatour/IMGDB/000012/000013/000589/00...,orange,SELB5BX8907A,17,18,12500
1,【秒殺！玩首爾】★不走人蔘護肝寶★明洞半天自由行、樂天超市、ＡＲ美術館、弘大藝術街、夜燈節５日,SELA5OZ8907D,團,報名,/eWeb_gloriatour/IMGDB/000012/000013/000589/00...,orange,SELA5OZ8907D,17,18,15400
2,【線上旅展】就去新加坡玩吧自由輕鬆走四日遊【送環球影城門票＋園區餐券】(二人成行),SIND4CI8907A,團,報名,/eWeb_gloriatour/IMGDB/000012/000014/000811/00...,orange,SIND4CI8907A,6,6,16500
3,【夏艷濟州】韓國濟州奇幻山坡、泰迪熊野生動物王國、透明獨木舟、樂高樂園、塗鴉秀５日,CJU05IT8907A,團,候補,/eWeb_gloriatour/IMGDB/000012/000013/000589/00...,orange,CJU05IT8907A,0,24,18500
4,【線上旅展】星光熠熠新加坡環球影城濱海灣花園金沙娛樂城超值四日(四人成行),SINI4CI8907A,團,報名,/eWeb_gloriatour/IMGDB/000012/000014/000811/00...,orange,SINI4CI8907A,6,6,21000
5,【馬新旺福】馬新雙國吉隆坡高塔自助餐馬六甲5D立體藝術館金沙娛樂城超值五日遊(新加坡一晚),KS205CI8907A,團,報名,/eWeb_gloriatour/IMGDB/000012/000014/000763/00...,orange,KS205CI8907A,17,18,21588
6,【線上旅展】馬來西亞沙巴長鼻猴螢河遊泛舟海洋國家公園五日(4+5星)(6人成行),BKIA5MH8907A,團,報名,/eWeb_gloriatour/IMGDB/000012/000014/000763/00...,orange,BKIA5MH8907A,8,8,25500
7,【米其林一星】食尚樂活新加坡環球影城S.E.A.海洋館濱海灣花園世界最高摩天輪四日【小孩不佔...,SINA4CI8907A,團,報名,/eWeb_gloriatour/IMGDB/000012/000014/000811/00...,orange,SINA4CI8907A,17,18,26300
8,菲航直飛菲常渴望巴拉望一島一飯店超級玩家五日遊(2人成行),PPSD5PR8907A,團,候補,/eWeb_gloriatour/IMGDB/000012/000014/000844/00...,orange,PPSD5PR8907A,0,20,26500
9,菲航直飛菲常渴望巴拉望新尊爵五日遊(兩晚渡假村)(2人成行),PPSA5PR8907A,團,候補,/eWeb_gloriatour/IMGDB/000012/000014/000844/00...,orange,PPSA5PR8907A,0,20,30900
